## <p style="color:blue">Commands to Setup DataBase</p>

### Download package information from all configured sources.

`> sudo apt update`


### Install MariaDB using below commands.

`> apt-get install software-properties-common`

`> apt-key adv --recv-keys --keyserver hkp://keyserver.ubuntu.com:80 0xF1656F24C74CD1D8`

`> add-apt-repository "deb [arch=amd64,arm64,ppc64el] http://mariadb.mirror.liquidtelecom.com/repo/10.4/ubuntu $(lsb_release -cs) main"`

`> apt-get update -y`

`> apt-get install mariadb-server -y`


### Cofigure MariaDB using the command below.

`> sudo mysql_secure_installation`

#### Use following settings for your configuration

```
Enter current password for root (enter for none): Provide your root user password
Switch to unix_socket authentication [Y/n] n
Change the root password? [Y/n] Y
New password:
Re-enter new password:
Remove anonymous users? [Y/n] Y
Disallow root login remotely? [Y/n] Y
Remove test database and access to it? [Y/n] Y
Reload privilege tables now? [Y/n] Y
```

### Access DataBase as root.
`> mysql -u root -p`

### Create New user *rajas1*

`> CREATE USER 'rajas1'@'localhost' IDENTIFIED BY 'Pass#123';`

### Grant all privilages to *rajas1*

`> GRANT ALL PRIVILEGES ON *.* TO 'rajas1'@'localhost' IDENTIFIED BY 'Pass#123';`

### Flush Privilages.

`> FLUSH PRIVILEGES;`

### Create new Database *C00265741*

`> CREATE DATABASE C00265741;`

### Use Database *C00265741* for futher operations.

`> USE C00265741;`

### Create New Table *Persons*

`CREATE TABLE C00265741.Persons(
    PrimeMinister varchar(255),
    Rank varchar(255),
    Party varchar(255),
    StartDate date,
    EndDate date,
    DOB date
);` 

### Create New Table *Monarchs*

`CREATE TABLE C00265741.Monarchs(
    Monarch varchar(255),
    StartDate date,
    EndDate date
    );` 

!pip install gazpacho
!pip install PyMySql
!pip install tabulate

In [26]:
import pandas as pd
from time import sleep
from datetime import datetime as dt
import re
import libDatabase as db                  #Script used as Library, build on PyMySql to insert data in Mysql Tables and handle exceptions. 
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from gazpacho import get, Soup

%matplotlib inline

#### Credentials for DB

In [27]:

host="localhost"
port=1234
user_name = "root"
password = "student"
schema_="C00265741"

#db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)


In [28]:
URL = "https://en.wikipedia.org/wiki/List_of_prime_ministers_of_the_United_Kingdom#18th_century"
URL_1 = "https://en.wikipedia.org"

soup = Soup(get(URL))
data = soup.find("table")
imp_data = data[1].find("tr")[2:]

In [29]:
rank = "Earl|Duke|Marquess|Lord|Viscount|Baron|Sir"

prime_minister = []
prime_minister_2 = []
prime_minister_1 = []
year_of_start = []
year_of_end = []
party = []
URL_S = []



#### Scraping PM, Start Date, End Date, Rank and links for wikipedia page of individual PM

In [30]:

i = 0
list_i = []
while i < len(imp_data)-1:
    #print (i)
    rowspan = imp_data[i].find("td")[2].attrs.get("rowspan")
    list_i.append(i)
    rowspan = int(rowspan)#if rowspan is not None else 1
    if type(imp_data[i].find("td")[2].find("a"))  is list:
        
        if imp_data[i].find("td")[2].find("a")[0].text == "Sir":
            #print("1sir")
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a")[0].attrs.get("title")+", " +imp_data[i].find("td")[2].find("a")[0].text]
            
        else:
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a")[0].attrs.get("title")]#* rowspan
        URL_S.append(URL_1 + imp_data[i].find("td")[2].find("a")[0].attrs.get("href"))
        #party = party + [imp_data[i].find("td")[6].text]
    
    else:
        
        URL_S.append(URL_1 + imp_data[i].find("td")[2].find("a").attrs.get("href"))
        if imp_data[i].find("td")[2].find("a").text == "Sir":
            #print("2sir", i)
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a").attrs.get("title") +", " + imp_data[i].find("td")[2].find("a").text]
        else:    
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a").attrs.get("title")]#* rowspan
        
        #party = party + [imp_data[i].find("td")[6].find("a").attrs.get("title")]#* rowspan
        #try:
            #party = party + [imp_data[i].find("td")[6].find("a").attrs.get("title")]#* rowspan
        #except:
            #party = party + [imp_data[i].find("td")[6].find("a")[0].attrs.get("title")]
            

    party.append(imp_data[i].find("td")[0].attrs.get("bgcolor"))
        
        
    if imp_data[i].find("td")[3].strip("br")[:-1] == "See also § Main articles:":
        
        year_of_start = year_of_start +[imp_data[i+1].find("td")[0].strip("br")] #* rowspan
        year_of_end = year_of_end+[imp_data[i+1].find("td")[1].strip("br")]
    else:
        year_of_start = year_of_start +[imp_data[i].find("td")[3].strip("br")] #* rowspan
        year_of_end = year_of_end+[imp_data[i].find("td")[4].strip("br")]
        
    i += rowspan
    


In [31]:
prime_minister_1 = [name.split(",")[0] for name in prime_minister]

for name in prime_minister:
    if len(name.split(",")) > 1:
        prime_minister_2.append(re.findall(rank, name.split(",")[1])[0])
    else:
        prime_minister_2.append("No Rank")

year_of_start = [dt.strptime(date_time, '%d %B%Y').strftime("%Y-%m-%d") for date_time in year_of_start]
year_of_end[-1] = dt.now().strftime("%d %B%Y")
year_of_end = [dt.strptime(date_time, '%d %B%Y').strftime("%Y-%m-%d") for date_time in year_of_end ]

In [32]:
new_labels = ['Whig', 'Tory', 'Conservative', 'Liberal', 'Labour']
old_labels = ["#FF7F00", "#3333CC", "#0087DC", "#ffd700", "#E4003B"]

df = pd.DataFrame({"Prime_Minister":prime_minister_1, "URL": URL_S,"Rank":prime_minister_2 ,"Party":party,"Year_of_start": year_of_start, "Year_of_end":year_of_end})
df['Party'] = df['Party'].replace(old_labels, new_labels)

df.head()


,Prime_Minister,URL,Rank,Party,Year_of_start,Year_of_end
0,Robert Walpole,https://en.wikipedia.org/wiki/Robert_Walpole,Sir,Whig,1721-04-03,1742-02-11
1,Spencer Compton,"https://en.wikipedia.org/wiki/Spencer_Compton,...",Earl,Whig,1742-02-16,1743-07-02
2,Henry Pelham,https://en.wikipedia.org/wiki/Henry_Pelham,No Rank,Whig,1743-08-27,1754-03-06
3,Thomas Pelham-Holles,https://en.wikipedia.org/wiki/Thomas_Pelham-Ho...,Duke,Whig,1754-03-16,1756-11-11
4,William Cavendish,https://en.wikipedia.org/wiki/William_Cavendis...,Duke,Whig,1756-11-16,1757-06-29


#### Looping through Wikipedia pages of every PM and scraping Date Of Brith

In [33]:
DOB = []
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]


for idx, URL in enumerate (df.iloc[:,1]):
#URL = df.iloc[1,1]

    soup = Soup(get(URL))
    data = soup.find("table")
    #print(df.iloc[idx,0],"--" , idx)

    for dat in data:
        if dat.attrs.get("class") == "infobox vcard":
            if type(dat.find("tr")) is list:
                for row in dat.find("tr"):
                    if row.find("th") is not None:
                        if type(row.find("th")) is list:
                            for new_rows in row.find("th"):
                                if new_rows.find("th").text == "Born":
                                    #print(df.iloc[idx,0], "-->",row[0].find("td").strip("br"))
                                    DOB.append(df.iloc[idx,0], "-->",row[0].find("td").strip("br"))

                        else:
                            
                            if row.find("th").text == "Born":
                                D_O_B =  row.find("td").strip("br")
                                if D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11][0].isalpha() == False:
                                    #print(D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11])
                                    DOB.append(D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11])
                                
                                else:
                                    if D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11] == "c. 1674Com":
                                        #print("1674-01-01")
                                        DOB.append("1674-01-01")
                                    for month in months:
                                        if D_O_B.find(month) > 0:
                                            #print(dt.strptime(D_O_B[D_O_B.find(month) + -2: D_O_B.find(month) +len(month) +5], '%d %B %Y').strftime('%Y-%m-%d'))
                                            DOB.append(dt.strptime(D_O_B[D_O_B.find(month) + -2: D_O_B.find(month) +len(month) +5], '%d %B %Y').strftime('%Y-%m-%d'))
                                     
            else:
                
                print(df.iloc[idx,1])
                #print (data)
                print("-----------------")
                DOB.append("NA")



        
        


In [34]:
df["Date_Of_Birth"] = DOB
#df["Date_Of_Death"] = DOD
#df_PM.to_csv("sample2.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Prime_Minister  77 non-null     object
 1   URL             77 non-null     object
 2   Rank            77 non-null     object
 3   Party           77 non-null     object
 4   Year_of_start   77 non-null     object
 5   Year_of_end     77 non-null     object
 6   Date_Of_Birth   77 non-null     object
dtypes: object(7)
memory usage: 4.3+ KB


In [35]:
i = 0 
dates = []
Run_Flag = True

while Run_Flag:
    #print(i)
    if type(imp_data[i].find("td")) is list:
        length = len(imp_data[i].find("td"))
        if length > 5:
            rowspan = imp_data[i].find("td")[-2].attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td")[-2].strip("p"))
        else:
            rowspan = imp_data[i].find("td")[-1].attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td")[-1].strip("p"))
            
    else:
        if imp_data[i].find("td") is not None:
            rowspan = imp_data[i].find("td").attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td").strip("p"))
        else:
            dates.append(imp_data[i].find("th")[-1].strip("p"))

            
        
        
    #dates.append(imp_data[i].find("td")[-2].strip("p").replace(imp_data[0].find("td")[-2].text, "").replace(" ", "").split("–"))
    #dates.append(imp_data[i].find("td")[-2].strip("p"))
    i += rowspan
    if dates[-1] == "Elizabeth II 1952–present":
        dates[-1] = dates[-1].replace("present", "2020")
        Run_Flag = False

#dates_1 = [dates.split(" ")[:2] for dates in dates]
dates[-3] += "–1936"
dates

['George I 1714–1727',
 'George II 1727–1760',
 'George III 1760–1820',
 'George IV 1820–1830',
 'William IV 1830–1837',
 'Victoria 1837–1901',
 'Edward VII 1901–1910',
 'George V 1910–1936',
 'Edward VIII 1936–1936',
 'George VI 1936–1952',
 'Elizabeth II 1952–2020']

In [36]:
monarch = list(map(lambda date: ''.join([i for i in date if i.isalpha()]), dates))
date_1 = list(map(lambda date: ''.join([i for i in date if i.isdigit()]), dates))
from_date = [date[:4] + "-01-01" for date in date_1]
to_date = [date[4:]+"-01-01" for date in date_1]

In [37]:
df_monarch = pd.DataFrame({"Monarch": monarch, "Start_Date": from_date, "End_Date":to_date})
#df_monarch.End_Date = pd.to_datetime(df_monarch.End_Date)
#df_monarch.Start_Date = pd.to_datetime(df_monarch.Start_Date)
df_monarch.to_csv("monarch.csv")

df_monarch.head()

,Monarch,Start_Date,End_Date
0,GeorgeI,1714-01-01,1727-01-01
1,GeorgeII,1727-01-01,1760-01-01
2,GeorgeIII,1760-01-01,1820-01-01
3,GeorgeIV,1820-01-01,1830-01-01
4,WilliamIV,1830-01-01,1837-01-01


#### Inserting Data into DB

In [38]:
from time import sleep
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
sleep(2)
for indx, rows in df.iterrows():
    db.insert_person(rows["Prime_Minister"], rows["Rank"], rows["Party"], rows["Year_of_start"], rows["Year_of_end"], rows["Date_Of_Birth"])

for indx, rows in df_monarch.iterrows():  
    db.insert_monarch(rows['Monarch'],rows['Start_Date'], rows['End_Date'] )
db.close_database()

### Q1. Which political party has produced the most individual prime ministers?

In [39]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
out = db.db_cmd_execute("select distinct Party from C00265741.Persons ;")
list_of_parties = [party[0] for party in out]
count_of_parties = []
for party in list_of_parties:
    count_of_parties.append(db.db_cmd_execute("select count(*) from C00265741.Persons where Party = '"+ str(party) +"' ;")[0][0])

print("Political party has produced the most individual prime ministers is: \033[1m", list_of_parties[count_of_parties.index(max(count_of_parties))], "\033[0m")
db.close_database()

Political party has produced the most individual prime ministers is:  Conservative 


### Q2. Provide a list of prime ministers who served under each monarch.

In [40]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
list_of_monarch = db.db_cmd_execute("select Monarch from C00265741.Monarchs ;")
list_of_monarch = [monarch[0] for monarch in list_of_monarch]
prime_ministers_monarch = {}
prime_ministers_monarch_1 = {}

for monarch in list_of_monarch:
    prime_ministers = db.db_cmd_execute("SELECT * FROM C00265741.Persons \
            where StartDate > (SELECT StartDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"') and \
            StartDate < (SELECT EndDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"')")
    prime_ministers_monarch[monarch] = [minister[0] for minister in prime_ministers]
    
    prime_ministers = db.db_cmd_execute("SELECT * FROM C00265741.Persons \
            where EndDate > (SELECT StartDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"') and \
            EndDate < (SELECT EndDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"')")
    prime_ministers_monarch_1[monarch] = [minister[0] for minister in prime_ministers]

    
for key in prime_ministers_monarch:
    temp = []
    print("\033[1m" +key+ "\033[0m", end = ' -->  \n')

    for name in prime_ministers_monarch[key]:
        temp.append(name)
        
    for name in prime_ministers_monarch_1[key]:
        temp.append(name)
        
    for name in set (temp):
        pass
        print(name)
    print("--"*50)
    
db.close_database()

GeorgeI -->  
Robert Walpole
----------------------------------------------------------------------------------------------------
GeorgeII -->  
Spencer Compton
Robert Walpole
William Cavendish
Henry Pelham
Thomas Pelham-Holles
----------------------------------------------------------------------------------------------------
GeorgeIII -->  
Spencer Perceval
William Petty
John Stuart
Henry Addington
William Pitt
Thomas Pelham-Holles
Charles Watson-Wentworth
Frederick North
George Grenville
Augustus FitzRoy
William Grenville
Robert Jenkinson
William Cavendish-Bentinck
William Pitt the Younger
----------------------------------------------------------------------------------------------------
GeorgeIV -->  
Robert Jenkinson
George Canning
F. J. Robinson
Arthur Wellesley
----------------------------------------------------------------------------------------------------
WilliamIV -->  
Charles Grey
William Lamb
Arthur Wellesley
Robert Peel
------------------------------------------------

### Q3. Which political party held the office of prime minister for the longest amount of overall time, and for how long (in years, months, and days)?

#### Function for converting Days to Year, Month and Days

In [41]:
def find_year( number_of_days ):
    if number_of_days is None:
        return(0, 0, 0)
    else:
        number_of_days = int(number_of_days)
        year = int(number_of_days / 365.25)
        months = int(round((number_of_days  % 365.25) / 30.436875, 0))
        days = int(round ((number_of_days  % 365.25) % 30.436875, 0))
        return(year, months, days)

In [42]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

number_of_days = []
for party in list_of_parties:
    term_start_end_dates = db.db_cmd_execute ("SELECT sum(datediff(EndDate, StartDate)) FROM C00265741.Persons where Party = '"+str(party)+"'")[0][0]
    number_of_days.append(term_start_end_dates)
duration = find_year(max(number_of_days))

print("Political party held the office of prime minister for the longest amount of overall time is: \033[1m", 
      list_of_parties[number_of_days.index(max(number_of_days))], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

Political party held the office of prime minister for the longest amount of overall time is:  Conservative  
i.e for  94  Years  6  Months and  16  Days


### Q4. Which individual politician held the office of prime minister for the longest amount of uninterrupted time, and for how long (in years, months, and days)?

In [43]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute ("select PrimeMinister, datediff(EndDate, StartDate) from C00265741.Persons \
                    order by datediff(EndDate, StartDate) desc limit 1;")[0]

duration = find_year(out[1])

print("Prime Minister held the office of prime minister for the longest amount of of uninterrupted time is: \033[1m",out[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

Prime Minister held the office of prime minister for the longest amount of of uninterrupted time is:  Robert Walpole  
i.e for  20  Years  10  Months and  10  Days


### Q5. Which individual politician has held the office of prime ministers for the longest amount of overall time?

In [44]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select distinct PrimeMinister from C00265741.Persons ;")
list_of_PrimeMinister = [minister[0] for minister in out]
list_of_overall_time = []
for minister in list_of_PrimeMinister:
    out = db.db_cmd_execute("SELECT  sum(datediff(EndDate, StartDate))  FROM C00265741.Persons \
                                where PrimeMinister = '"+ str(minister)+"';")[0][0]
    list_of_overall_time.append(out)
    
df_temp = pd.DataFrame({"PM":list_of_PrimeMinister,"time":list_of_overall_time})
overall_longest = df_temp[df_temp.time == df_temp.time.max()].values[0]  
duration = find_year(int(overall_longest[1]))

print("Politician has held the office of prime ministers for the longest amount of overall time: \033[1m",overall_longest[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()



Politician has held the office of prime ministers for the longest amount of overall time:  Robert Walpole  
i.e for  20  Years  10  Months and  10  Days


### Q6. Which individual politician held the office of prime minister for the shortest amount of overall time, and for how long (in years, months, and days)?

In [45]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select distinct PrimeMinister from C00265741.Persons ;")
list_of_PrimeMinister = [minister[0] for minister in out]
list_of_overall_time = []
for minister in list_of_PrimeMinister:
    out = db.db_cmd_execute("SELECT  sum(datediff(EndDate, StartDate))  FROM C00265741.Persons \
                                where PrimeMinister = '"+ str(minister)+"';")[0][0]
    list_of_overall_time.append(out)
    
df_temp = pd.DataFrame({"PM":list_of_PrimeMinister,"time":list_of_overall_time})
overall_shortest = df_temp[df_temp.time == df_temp.time.min()].values[0]  
duration = find_year(int(overall_shortest[1]))

print("Politician has held the office of prime ministers for the shortest amount of overall time: \033[1m",overall_shortest[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()


Politician has held the office of prime ministers for the shortest amount of overall time:  George Canning  
i.e for  0  Years  4  Months and  27  Days


### Q7. List the prime ministers who where either an earl, viscount, duke, or knight?

In [46]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

ministers = db.db_cmd_execute("select PrimeMinister,Rank from C00265741.Persons \
                                where Rank = 'earl' or Rank = 'viscount' or Rank = 'duke' or \
                                Rank = 'Sir' order by Rank;")
print(tabulate(set(ministers),headers= ["Prime Minister", "Rank"], tablefmt="psql"))
    
db.close_database()

+----------------------------+----------+
| Prime Minister             | Rank     |
|----------------------------+----------|
| John Russell               | Earl     |
| William Lamb               | Viscount |
| Edward Smith-Stanley       | Earl     |
| Arthur Wellesley           | Duke     |
| William Cavendish-Bentinck | Duke     |
| John Stuart                | Earl     |
| Augustus FitzRoy           | Duke     |
| Winston Churchill          | Sir      |
| William Pitt               | Earl     |
| F. J. Robinson             | Viscount |
| Henry John Temple          | Viscount |
| William Cavendish          | Duke     |
| William Petty              | Earl     |
| Spencer Compton            | Earl     |
| Thomas Pelham-Holles       | Duke     |
| Archibald Primrose         | Earl     |
| Robert Peel                | Sir      |
| Robert Walpole             | Sir      |
| Charles Grey               | Earl     |
| Alec Douglas-Home          | Sir      |
| Henry Campbell-Bannerman   | Sir

### Q8. What age was each prime minister on the day they assumed office?

In [47]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select PrimeMinister, StartDate, datediff(StartDate, DOB) from C00265741.Persons;")
out = [list(row[0:2])+ list(find_year(row[2])) for row in out]

print(tabulate(out, tablefmt="fancy_grid", headers=["Prime Minister", "Date of Assuming", "Age(Years)", "Age(Month)", "Age(Days)"]))  
db.close_database()

╒════════════════════════════╤════════════════════╤══════════════╤══════════════╤═════════════╕
│ Prime Minister             │ Date of Assuming   │   Age(Years) │   Age(Month) │   Age(Days) │
╞════════════════════════════╪════════════════════╪══════════════╪══════════════╪═════════════╡
│ Robert Walpole             │ 1721-04-03         │           44 │            7 │           6 │
├────────────────────────────┼────────────────────┼──────────────┼──────────────┼─────────────┤
│ Spencer Compton            │ 1742-02-16         │           68 │            1 │          15 │
├────────────────────────────┼────────────────────┼──────────────┼──────────────┼─────────────┤
│ Henry Pelham               │ 1743-08-27         │           48 │           11 │           0 │
├────────────────────────────┼────────────────────┼──────────────┼──────────────┼─────────────┤
│ Thomas Pelham-Holles       │ 1754-03-16         │           60 │            8 │          24 │
├────────────────────────────┼──────────

### Q9. On the last day of their term of office, which politician was the oldest?

In [48]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select PrimeMinister, EndDate, datediff(EndDate, DOB) from C00265741.Persons;")

    
df_temp = pd.DataFrame(out)
age = df_temp.loc[df_temp[2] == df_temp[2].max(), :].values[0]
duration = find_year(int(age[2]))

print("Oldest Politician on the last day of the term: \033[1m",age[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

Oldest Politician on the last day of the term:  William Ewart Gladstone  
i.e for  84  Years  2  Months and  2  Days
